In [ ]:
import sys
sys.path.append('..')
print(sys.path)
import yaml
import matplotlib.pyplot as plt
from wrappers.data_wrapper import *
from wrappers.suite2p_wrapper import *
from wrappers.args_check import args_check

### file convert

In [ ]:
image = ImageData('../../backend/files/Sue_2x_3000_40_-46.tif')
with open('../config/suite2p_file_convert.yaml') as f:
    params = yaml.safe_load(f)
file_info = suite2p_file_convert(image, params)

In [ ]:
file_info['ops'].data['meanImg'].shape

In [ ]:
tmp = np.load(file_info['ops'].data['ops_path'], allow_pickle=True)
tmp

### registration 

In [ ]:
@args_check
def suite2p_registration(
        ops: Suite2pData=None, params: dict=None
    ) -> {'ops': Suite2pData}:
    # refImg = image.data
    ops = ops.data
    refImg = ops['meanImg']

    import numpy as np
    from suite2p import registration
    from suite2p import default_ops

    ######### REGISTRATION #########
#     if len(refImg.shape) == 3:
#         refImg = refImg[0]

    # if ops is None:
    #     ops = {**default_ops()}

    ops = {**ops, **params}

    ops = registration.register_binary(ops, refImg=refImg) # register binary

    # compute metrics for registration
    if ops.get('do_regmetrics', True) and ops['nframes']>=1500:
        ops = registration.get_pc_metrics(ops)

    info = {}
    info['images'] = ImageData(ops['refImg'].astype(np.uint8), func_name='suite2p_registration', file_name='refImg')
    info['meanImgE'] = ImageData(ops['meanImgE'].astype(np.uint8), func_name='suite2p_registration', file_name='meanImgE')
    info['ops'] = Suite2pData(ops, func_name='suite2p_registration', file_name='ops')

    return info

In [ ]:
with open('../config/suite2p_registration.yaml') as f:
    params = yaml.safe_load(f)
regist_info = suite2p_registration(*file_info.values(), params=params)

In [ ]:
regist_info['ops'].data['reg_file']

In [ ]:
from suite2p import io
reg_file = io.BinaryFile(
    read_filename='./files/suite2p/plane0/data.bin', 
    Ly=170, 
    Lx=170
).data
                                 

In [ ]:
plt.imshow(reg_file[0])

In [ ]:
regist_info

In [ ]:
regist_info['ops'].data['regDX'].shape

In [ ]:
plt.plot(regist_info['ops'].data['corrXY1'])

### ROI

In [ ]:
roi_info = suite2p_roi(*file_info.values(), params)

### spike deconv

In [ ]:
with open('../config/suite2p_spike_deconv.yaml') as f:
    params = yaml.safe_load(f)
spike_info = suite2p_spike_deconv(*roi_info.values(), params)

In [ ]:
roi_info['ops'].data

In [ ]:
from suite2p.extraction import dcnv
tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with␣

# gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters
prctile_baseline = 8
batch_size = 200
ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff, 'baseline': baseline, 'sig_baseline': sig_baseline, 
       'win_baseline': win_baseline, 'prctile_baseline': prctile_baseline, 'batch_size': batch_size}


neucoeff = 0.7
Fc = roi_info['F'].data - ops['neucoeff'] * roi_info['ops'].data['Fneu']

# baseline operation
Fc = dcnv.preprocess(
    F=Fc,
    baseline=ops['baseline'],
    win_baseline=ops['win_baseline'],
    sig_baseline=ops['sig_baseline'],
    fs=ops['fs'],
    prctile_baseline=ops['prctile_baseline']
)

# get spikes
spks = dcnv.oasis(F=Fc, batch_size=ops['batch_size'], tau=ops['tau'], fs=ops['fs'])

In [ ]:
print(roi_info['F'].data.shape)
print(roi_info['ops'].data['Fneu'].shape)

In [ ]:
spks.shape

In [ ]:
plt.imshow(roi_info['ops'].data[ 'meanImg'])

In [ ]:
plt.imshow(regist_info['ops'].data[ 'meanImgE'])

In [ ]:
from pathlib import Path
# iscell = np.load('./files/suite2p/plane0/iscell.npy', allow_pickle=True)[:, 0].astype(bool)
stats = np.load('./files/suite2p/plane0/stat.npy', allow_pickle=True)
stats.shape, iscell.shape